<a href="https://colab.research.google.com/github/KuvinaDesch/MAT421/blob/main/ModC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finding the roots of a function is one of the most important things you can do with it.  A root is wherever the function value is equal to zero.  By adding a constant to the function, finding roots can become finding any output you want to look for.  Some functions, like linear and quadratic ones, have really simple formulas to find the exact roots.  Not all functions are like this though, so we want a general method.  Python's scipy has an easy way to do this

In [14]:
import numpy as np
from scipy import optimize

# defining a function for which we want the root
def f(x):
  return np.sin(x) + x**2 - 1

# find root using scipy, starting the method at x=0
root = optimize.fsolve(f, 0)

# the root is found, and inputting it into the original function
# gives us basically zero
print(root)
print(f(root))


[0.63673265]
[-1.11022302e-16]


In [15]:
import numpy as np
from scipy import optimize

# if we use a function that has no root
def f(x):
  return np.e**x

# the method will stop once it realizes it's making no progress
root = optimize.fsolve(f, 0)

# we're stuck with whatever it last checked
print(root)
print(f(root))


[-276.10007928]
[1.23384077e-120]


Tolerance is an important concept.  When looking for roots, the exact values are not always known.  Our measured value may be different than the actual value by some amount.  This is the error.  Tolerance is the maximum error that you will allow.  When a zero is found within the tolerance, it means the program has converged.  When we find a root x_r, we can test how close it is to zero with |f(x_r)|.  Once this is smaller than the tolerance, we stop looking because it has converged.  Sometimes a function has no roots, but it has points that are so close to zero, that they're within the tolerance.

A very simple root finding method is the bisection method.  The mean value theorem tells you that if you have 2 points (x_1,y_1), (x_2,y_2) of a continuous function f, then for any value y_3 between y_1 and y_2, you can find a value x_3 between x_1 and x_2 such that f(x_3) = y_3.  Applying this to root finding, if y_1 is positive and y_2 is negative (or vice versa), then there must be a root between x_1 and x_2.  The midpoint will be your guess and if it's a root (or within the tolerance), you're done, but if not, you have cut the search range in half.

We will implement such a method in python.

In [31]:
import numpy as np

# given the function f, 2 starting points a and b, and tolerance
def findRoot(f, a, b, tolerance):

  # if a and b have matching signs, a root is not guaranteed, so we stop
  if(np.sign(f(a)) == np.sign(f(b))):
    raise Exception("a and b have the same sign, so a root is not guaranteed")
  else:

    # otherwise, calculate the midpoint.
    # we also print it after each step to see how the method works
    mid = (a+b)/2
    print(mid)

    # if the midpoint is a root, we return it and stop
    if (np.abs(f(mid)) < tolerance):
      return mid
    else:

      # otherwise we use it for another round because this function is recursive
      # if its sign matches a, we replace a with mid
      # otherwise, its sign matches b, so we replace b with mid
      if (np.sign(f(mid)) == np.sign(f(a))):
        return findRoot(f, mid, b, tolerance)
      else:
        return findRoot(f, a, mid, tolerance)




# x^5 - 2x is our function
def f(x):
  return x**5 - 2*x

# it finds a root
root = findRoot(f, 1, 2, 0.0001)
print("root:")
print (root)
print("f(root):")
print (f(root))

1.5
1.25
1.125
1.1875
1.21875
1.203125
1.1953125
1.19140625
1.189453125
1.1884765625
1.18896484375
1.189208984375
root:
1.189208984375
f(root):
1.4955037002550853e-05


The bisection method is simple, but it can be slow.  A more efficient method is the Newton-Raphson method.  It's really easy to find roots for linear functions, but most functions aren't linear.  But a core idea of calculus is instantaneous rate of change, or the slope at a single point.  If you use a linear function with this slope, it simulates the main function pretty well on a local scale.  We can use this recursively and it find the root pretty fast.  It only works on functions that are smooth (have a continuous derivative).  

Given a value x0, the linear approximation around that point is calculated with f(x) ~= f(x0) + f'(x0)(x-x0)  

To find a zero, we set this equal to zero and solve for x.  That results in

x = x0 -f(x0)/f'(x0)

This x will become our new x0 when we repeat the algorithm.

In [36]:
import numpy as np

# given function f, derivative fp, starting guess x0, and tolerance
def newtonRaphson(f, fp, x0, tol):
  
  # find the next guess with the formula
  x = x0 - (f(x0)/fp(x0))

  # if it's within the tolerance, we're done
  if (np.abs(f(x)) < tol):
    return x

  # otherwise, we use it as the next guess and use the function recursively
  else:
    return newtonRaphson(f, fp, x, tol)

# solving 0 = x^2 - 2 approximates the square root of 2
def f(x):
  return x**2 - 2

# the derivative of x^2 is 2x
def fp(x):
  return 2*x

# with a starting guess of 1.5 and tolerance of 0.00001, we get very close
root = newtonRaphson(f, fp, 1.5, 0.00001)
print(root)
print(root**2)




1.4142156862745099
2.000006007304883


The problem with this method is that you need the derivative and it can also run into issues.  If the slope is exactly 0 at your guess, then there is no root to the linear approximation.  And if the slope is close to 0, it can throw you off by a lot.

Instead of doing things manually, we can just use scipy's fsolve

In [41]:
import numpy as np
from scipy.optimize import fsolve


def f(x):
  return 2*x**3 - x

# we can use a list of 4 starting guesses and it gives us
# the result from each one in a list
x = fsolve(f, [-1, 0, 1, 2])
print(x)

[-0.70710678  0.          0.70710678  0.70710678]
